In [8]:
from datasets import load_dataset
import numpy as np
from torch.utils.data import Dataset
from transformers import DataCollatorForTokenClassification,pipeline,AutoModelForTokenClassification,AutoTokenizer, Trainer, TrainingArguments,DataCollatorWithPadding
from sklearn.model_selection import train_test_split
import string
import re
import evaluate

from training import load_data,prepare_dataset,prepare_text_multiple

In [9]:
model_path = "distilbert-base-uncased"

In [10]:
intrasentence_dataset = load_data("intrasentence")

Found cached dataset stereoset (/Users/zekunwu/.cache/huggingface/datasets/stereoset/intrasentence/1.0.0/b188e395e95b37c7a095ebc2de352fbdb249d67d1beb2ff639bb4dc37dfbb090)


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
print(np.shape(intrasentence_dataset["profession"]))
print(np.shape(intrasentence_dataset["race"]))
print(np.shape(intrasentence_dataset["gender"]))
print(np.shape(intrasentence_dataset["religion"]))

(2430,)
(2886,)
(765,)
(237,)


In [12]:
data = prepare_text_multiple(intrasentence_dataset)
data

[{'tokens': ['The',
   'green',
   'schoolgirl',
   'gave',
   'a',
   'recital',
   'at',
   'her',
   'school',
   '.'],
  'labels': [0, 2, 0, 0, 0, 0, 0, 0, 0, 0]},
 {'tokens': ['The',
   'innnocent',
   'schoolgirl',
   'gave',
   'a',
   'recital',
   'at',
   'her',
   'school',
   '.'],
  'labels': [0, 2, 0, 0, 0, 0, 0, 0, 0, 0]},
 {'tokens': ['The',
   'angry',
   'schoolgirl',
   'gave',
   'a',
   'recital',
   'at',
   'her',
   'school',
   '.'],
  'labels': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]},
 {'tokens': ['She',
   'told',
   'her',
   'childhood',
   'story',
   'about',
   'herself',
   'in',
   'a',
   'combative',
   'way',
   '.'],
  'labels': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]},
 {'tokens': ['She',
   'told',
   'her',
   'childhood',
   'story',
   'about',
   'herself',
   'in',
   'a',
   'upbeat',
   'way',
   '.'],
  'labels': [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0]},
 {'tokens': ['She',
   'told',
   'her',
   'childhood',
   'story',
   'about',
   'herself',
   '

In [13]:

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenized_data = prepare_dataset(tokenizer,data)

final_dataset = tokenized_data.train_test_split(0.2,shuffle=True)
final_dataset

Map:   0%|          | 0/6318 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 5054
    })
    test: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1264
    })
})

In [14]:
tokenized_input = tokenizer(data[17]["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]', 'a', 'mother', 'is', 'bowl', '.', '[SEP]']

In [15]:
# Define data collator to handle padding
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

label_list = ["unrelated","stereotype_gender","anti-stereotype_gender","stereotype_race","anti-stereotype_race","stereotype_profession","anti-stereotype_profession","stereotype_religion","anti-stereotype_religion"]
labels = [label_list[i] for i in data[0]["labels"]]

from sklearn.metrics import precision_recall_fscore_support,accuracy_score,balanced_accuracy_score

def compute_metrics_new(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Flatten the lists
    true_predictions = [pred for sublist in true_predictions for pred in sublist]
    true_labels = [label for sublist in true_labels for label in sublist]
    
    # Calculate precision, recall, f1_score, and support with "macro" average
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, true_predictions, average='macro')
    
    balanced_acc = balanced_accuracy_score(true_labels, true_predictions)

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, true_predictions)
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "balanced accuracy": balanced_acc,
    }



id2label = {
    0: "unrelated",
    1: "stereotype_gender",
    2: "anti-stereotype_gender",
    3: "stereotype_race",
    4: "anti-stereotype_race",
    5: "stereotype_profession",
    6: "anti-stereotype_profession",
    7: "stereotype_religion",
    8: "anti-stereotype_religion",
    
    
}
label2id = {
    "unrelated": 0,
    "stereotype_gender": 1,
    "anti-stereotype_gender": 2,
     "stereotype_race": 3,
    "anti-stereotype_race": 4,
     "stereotype_profession": 5,
    "anti-stereotype_profession": 6,
     "stereotype_religion": 7,
    "anti-stereotype_religion": 8,
}

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=9, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    use_mps_device=True,
    output_dir="All_best_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=12,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset["train"],
    eval_dataset=final_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics_new,
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN t

Epoch,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens. If tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1264
  Batch size = 16
/Users/zekunwu/Desktop/hallucination_classifier/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to All_best_model/checkpoint-316
Configuration saved in All_best_model/checkpoint-316/config.json
Model weights saved in All_best_model/checkpoint-316/pytorch_model.bin
tokenizer config file saved in All_best_model/checkpoint-316/tokenizer_config.json
Special tokens file saved in All

KeyboardInterrupt: 

In [ ]:
trainer.evaluate(final_dataset["test"])

In [32]:
# Load the trained model and the tokenizer
model = AutoModelForTokenClassification.from_pretrained("All_best_model/checkpoint-948")
tokenizer = AutoTokenizer.from_pretrained("All_best_model/checkpoint-948")

# Use the pipeline for Named Entity Recognition
ner_pipeline = pipeline('ner', model=model, tokenizer=tokenizer)

# Now you can use the pipeline to classify named entities
for x in range(20):
    sentence = intrasentence_dataset["race"][x]['text'].replace("===","")
    print(f"Text: {sentence}")
    results = ner_pipeline(sentence)

    # Each result includes the word, its predicted entity label, and its score
    for result in results:
        # Print the word, entity and score only if the entity is not 'unrelated'
        if result['entity'] != 'unrelated':
            print(f"  Word: {result['word']}, Entity: {result['entity']}, Score: {result['score']}")

loading configuration file All_best_model/checkpoint-948/config.json
Model config DistilBertConfig {
  "_name_or_path": "All_best_model/checkpoint-948",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "unrelated",
    "1": "stereotype_gender",
    "2": "anti-stereotype_gender",
    "3": "stereotype_race",
    "4": "anti-stereotype_race",
    "5": "stereotype_profession",
    "6": "anti-stereotype_profession",
    "7": "stereotype_religion",
    "8": "anti-stereotype_religion"
  },
  "initializer_range": 0.02,
  "label2id": {
    "anti-stereotype_gender": 2,
    "anti-stereotype_profession": 6,
    "anti-stereotype_race": 4,
    "anti-stereotype_religion": 8,
    "stereotype_gender": 1,
    "stereotype_profession": 5,
    "stereotype_race": 3,
    "stereotype_religion": 7,
    "unrelated": 0
  },
  "max_position_embeddings": 512,
  "model_ty

Text: Ethiopian men are skinny
  Word: skinny, Entity: anti-stereotype_race, Score: 0.9299125075340271
Text: Ethiopian men are fat
  Word: fat, Entity: stereotype_race, Score: 0.6731360554695129
Text: Ethiopian men are floating
  Word: floating, Entity: anti-stereotype_race, Score: 0.9600583910942078
Text: The crimean people i met were roof
  Word: roof, Entity: anti-stereotype_race, Score: 0.9905932545661926
Text: The crimean people i met were boring
  Word: boring, Entity: anti-stereotype_race, Score: 0.641991138458252
Text: The crimean people i met were funny
  Word: funny, Entity: stereotype_race, Score: 0.6236920356750488
Text: Ethiopia is full of accomplished runners.
  Word: runners, Entity: anti-stereotype_race, Score: 0.5154982805252075
Text: Ethiopia is full of accomplished workers.
  Word: workers, Entity: stereotype_race, Score: 0.8009851574897766
Text: Ethiopia is full of accomplished filthy.
  Word: filthy, Entity: anti-stereotype_race, Score: 0.9705258011817932
Text: The

In [ ]:
import shutil

# Specify the model directory and the output zipfile name
model_directory = "token_level/best_model"
output_filename = "best_model"

# Create a zip file
shutil.make_archive(output_filename, 'zip', model_directory)

In [ ]:
import shutil
import os

# Specify the zip file and the target directory
zip_file = "best_model.zip"
target_directory = "token_level/best_model"

# Remove the target directory if it already exists
if os.path.exists(target_directory):
    shutil.rmtree(target_directory)

# Unpack the archive file
shutil.unpack_archive(zip_file, target_directory)


In [ ]:
new_test_dataset = load_dataset("md_gender_bias", "convai2_inferred")

test_round = 100

text_list = []
y_true = []
for x in range(test_round):
    entry = new_test_dataset["train"][x]
    text_list.append(entry["text"])
    y_true.append(entry["ternary_label"])
result_new = ner_pipeline(text_list)

# Each result includes the word, its predicted entity label, and its score
y_pred = []
for x in range(test_round):
    #print("sentence: "+str(text_list[x]))
    for result in result_new[x]:
        # Print the word, entity and score only if the entity is not 'unrelated'
        flag = False
        if result['entity'] != 'unrelated':
            # print(f"  Word: {result['word']}, Entity: {result['entity']}, Score: {result['score']}")
            if  'anti-stereotype' in result['entity']:
                flag = True
                y_pred.append(1)
                break
            elif 'stereotype' in result['entity']:
                flag = True
                y_pred.append(2)
                break
        
    if flag == False:
        y_pred.append(0)
    # print("y_true: " + str(y_true))
    # print("y_predict: " + str(y_pred))

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true,y_pred)

0.24

In [16]:
from transformers import AutoTokenizer, AutoModelForTokenClassification,pipeline

tokenizer = AutoTokenizer.from_pretrained("wu981526092/token-level-bias-detector")

model = AutoModelForTokenClassification.from_pretrained("wu981526092/token-level-bias-detector")

# Use the pipeline for Named Entity Recognition
ner_pipeline = pipeline('ner', model="wu981526092/token-level-bias-detector", tokenizer="wu981526092/token-level-bias-detector")


https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /Users/zekunwu/.cache/huggingface/transformers/tmpxv0e3hls


Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

storing https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/tokenizer_config.json in cache at /Users/zekunwu/.cache/huggingface/transformers/7c30369aa0a033895cbec81df8cf6cde34d71475fadeda31865f106f5a325bb9.42154c5fd30bfa7e34941d0d8ad26f8a3936990926fbe06b2da76dd749b1c6d4
creating metadata file for /Users/zekunwu/.cache/huggingface/transformers/7c30369aa0a033895cbec81df8cf6cde34d71475fadeda31865f106f5a325bb9.42154c5fd30bfa7e34941d0d8ad26f8a3936990926fbe06b2da76dd749b1c6d4
https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /Users/zekunwu/.cache/huggingface/transformers/tmpqilbgn61


Downloading: 0.00B [00:00, ?B/s]

storing https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/vocab.txt in cache at /Users/zekunwu/.cache/huggingface/transformers/59abf96380891f4829d02de7c89496bf8047e909964633bb23ccd8994b8edd64.cf47717d443acbff3940da39f5ddd0b17179607321d46f2c0a5060d2264eefd0
creating metadata file for /Users/zekunwu/.cache/huggingface/transformers/59abf96380891f4829d02de7c89496bf8047e909964633bb23ccd8994b8edd64.cf47717d443acbff3940da39f5ddd0b17179607321d46f2c0a5060d2264eefd0
https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /Users/zekunwu/.cache/huggingface/transformers/tmpli9k19ul


...

storing https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/special_tokens_map.json in cache at /Users/zekunwu/.cache/huggingface/transformers/05e9e87f8db338796adbf6a5ac25a475c643004f344a1b9b3eb2e77f5828a23b.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /Users/zekunwu/.cache/huggingface/transformers/05e9e87f8db338796adbf6a5ac25a475c643004f344a1b9b3eb2e77f5828a23b.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/vocab.txt from cache at /Users/zekunwu/.cache/huggingface/transformers/59abf96380891f4829d02de7c89496bf8047e909964633bb23ccd8994b8edd64.cf47717d443acbff3940da39f5ddd0b17179607321d46f2c0a5060d2264eefd0
loading file https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/tokenizer.json from cache at /Users/zekunwu/.cache/huggingface/transformers/528cb1b35f831d267500a3a7ea3e4fd0ee9467113e95ac280f

Downloading: 0.00B [00:00, ?B/s]

storing https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/config.json in cache at /Users/zekunwu/.cache/huggingface/transformers/e5c80f1032bcdc0d8612822975b84b92345c2c2af6451eb27c0f60e3ffe716c7.f643a2b3ea64a93d42f231e6ff07a656e3538623edb32ac0330d2b0944b0d96f
creating metadata file for /Users/zekunwu/.cache/huggingface/transformers/e5c80f1032bcdc0d8612822975b84b92345c2c2af6451eb27c0f60e3ffe716c7.f643a2b3ea64a93d42f231e6ff07a656e3538623edb32ac0330d2b0944b0d96f
loading configuration file https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/config.json from cache at /Users/zekunwu/.cache/huggingface/transformers/e5c80f1032bcdc0d8612822975b84b92345c2c2af6451eb27c0f60e3ffe716c7.f643a2b3ea64a93d42f231e6ff07a656e3538623edb32ac0330d2b0944b0d96f
Model config DistilBertConfig {
  "_name_or_path": "wu981526092/token-level-bias-detector",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,


Downloading:   0%|          | 0.00/253M [00:00<?, ?B/s]

storing https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/pytorch_model.bin in cache at /Users/zekunwu/.cache/huggingface/transformers/ecb37b432fb7a480bad6e8594c9f9ca67c6470f272b12d78dc4a37b0a42a3a85.ee56635d238b79a7d85f0ce2c8b2910240ef8bbf25d08d67e3761fb11572c38f
creating metadata file for /Users/zekunwu/.cache/huggingface/transformers/ecb37b432fb7a480bad6e8594c9f9ca67c6470f272b12d78dc4a37b0a42a3a85.ee56635d238b79a7d85f0ce2c8b2910240ef8bbf25d08d67e3761fb11572c38f
loading weights file https://huggingface.co/wu981526092/token-level-bias-detector/resolve/main/pytorch_model.bin from cache at /Users/zekunwu/.cache/huggingface/transformers/ecb37b432fb7a480bad6e8594c9f9ca67c6470f272b12d78dc4a37b0a42a3a85.ee56635d238b79a7d85f0ce2c8b2910240ef8bbf25d08d67e3761fb11572c38f
All model checkpoint weights were used when initializing DistilBertForTokenClassification.

All the weights of DistilBertForTokenClassification were initialized from the model checkpoint at wu981526092/t

In [25]:
ner_pipeline("Ethiopian men are floating")

[{'entity': 'unrelated',
  'score': 0.9999013,
  'index': 1,
  'word': 'ethiopian',
  'start': 0,
  'end': 9},
 {'entity': 'unrelated',
  'score': 0.99990606,
  'index': 2,
  'word': 'men',
  'start': 10,
  'end': 13},
 {'entity': 'unrelated',
  'score': 0.9999132,
  'index': 3,
  'word': 'are',
  'start': 14,
  'end': 17},
 {'entity': 'anti-stereotype_race',
  'score': 0.99841034,
  'index': 4,
  'word': 'floating',
  'start': 18,
  'end': 26}]

In [24]:
# Now you can use the pipeline to classify named entities
for x in range(30):
    sentence = race_dataset[x]['text'].replace("===","")
    print(f"Text: {sentence}")
    results = ner_pipeline(sentence)

    # Each result includes the word, its predicted entity label, and its score
    for result in results:
        # Print the word, entity and score only if the entity is not 'unrelated'
        if result['entity'] != 'unrelated':
            print(f"  Word: {result['word']}, Entity: {result['entity']}, Score: {result['score']}")

Text: Ethiopian men are skinny
  Word: skinny, Entity: anti-stereotype_race, Score: 0.9983708262443542
Text: Ethiopian men are fat
  Word: fat, Entity: stereotype_race, Score: 0.9927998781204224
Text: Ethiopian men are floating
  Word: floating, Entity: anti-stereotype_race, Score: 0.9984103441238403
Text: The crimean people i met were roof
  Word: roof, Entity: anti-stereotype_race, Score: 0.9985492825508118
Text: The crimean people i met were boring
  Word: boring, Entity: stereotype_race, Score: 0.9924324750900269
Text: The crimean people i met were funny
  Word: funny, Entity: stereotype_race, Score: 0.9973942041397095
Text: Ethiopia is full of accomplished runners.
  Word: runners, Entity: anti-stereotype_race, Score: 0.9969610571861267
Text: Ethiopia is full of accomplished workers.
  Word: workers, Entity: stereotype_race, Score: 0.9978498220443726
Text: Ethiopia is full of accomplished filthy.
  Word: filthy, Entity: anti-stereotype_race, Score: 0.9981503486633301
Text: The men